In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import scipy
from PIL import Image
from scipy import ndimage

In [3]:
import gdal
import os
import geopandas as gpd
from skimage import io
from skimage.io import imread
%matplotlib inline

# first check for one image if it works then make a for loop and apply to all images

# reading the input image 10_2019, and extracting the nan indices form it

In [4]:
image_test = gdal.Open("E:\\Internship_Harvesting\\Dataset\\10_2019.tif")
image_test

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F8F647A50> >

In [5]:
image_test.RasterYSize, image_test.RasterXSize, image_test.RasterCount

(8963, 8298, 5)

In [7]:
image_zero = np.zeros((image_test.RasterYSize, image_test.RasterXSize, image_test.RasterCount))

In [8]:
image_zero.shape

(8963, 8298, 5)

In [10]:
for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = image_test.GetRasterBand(b + 1).ReadAsArray()

# extracting nan indices for one band of the image 10_2019

In [11]:
I1B1 = image_zero[:,:,0]
I1B1.shape

(8963, 8298)

In [14]:
np.count_nonzero(np.isnan(I1B1)), np.count_nonzero(~np.isnan(I1B1))

(35318365, 39056609)

In [44]:
I1B1_nan_index = np.argwhere(np.isnan(I1B1))

# once we have the nan indices, we will map these indices in polytoras image and make corresponding values as NaN.
## here polytoras is the image obtained from converting the .shp file consisting of polygon to .tif file using QGIS software 

In [37]:
polytoras = gdal.Open("E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\polytoras.tif")
polytoras

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F8EB43120> >

In [38]:
polytoras.RasterYSize, polytoras.RasterXSize, polytoras.RasterCount

(8963, 8298, 1)

In [40]:
polytoras_array = polytoras.GetRasterBand(1).ReadAsArray()

In [42]:
polytoras_array.shape

(8963, 8298)

In [46]:
I1B1_nan_index.shape

(35318365, 2)

In [47]:
for i in range(I1B1_nan_index.shape[0]):
    polytoras_array[I1B1_nan_index[i,0], I1B1_nan_index[i,1]] = np.NaN

In [49]:
np.unique(polytoras_array)

array([ 0.,  2., nan, ..., nan, nan, nan], dtype=float32)

# now extract all the zero value indices from polytoras_array and store that in some numpy array
## here zero value indices in polytoras image are non-agriculture points, which must be masked

In [52]:
polytoras_0_index = np.argwhere((polytoras_array == 0.))

In [56]:
polytoras_0_index.shape

(5256544, 2)

In [61]:
polytoras_0_index

array([[   0, 5731],
       [   0, 5732],
       [   0, 5733],
       ...,
       [8961, 5785],
       [8961, 5792],
       [8961, 5793]], dtype=int64)

# now make all the values corresponding the indices polytoras_0_index to NaN in the input image 10_2019, thus 10_2019 image will have 40574909 NaN values in total, 35318365 (outside the boundary) plus 5256544(non-agriculture inside the boundary)
# non-nan pixels would belong to agriculture part which needs to be classify in wheat and non-wheat

In [66]:
for i in range(polytoras_0_index.shape[0]):
    I1B1[polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN

In [68]:
np.count_nonzero(np.isnan(I1B1)), np.count_nonzero(~np.isnan(I1B1)) 

(40574909, 33800065)

# at this poin I1B1 has all pixel values as NaN for non-agriculture and outside map, it has numeric pixel values only for agriculture part

# this process has to done on all the bands of each image, trying to making a for loop

In [70]:
image = ["E:\\Internship_Harvesting\\Dataset\\10_2019.tif",
        "E:\\Internship_Harvesting\\Dataset\\11_2019.tif",
        "E:\\Internship_Harvesting\\Dataset\\12_2019.tif",
        "E:\\Internship_Harvesting\\Dataset\\01_2020.tif",
        "E:\\Internship_Harvesting\\Dataset\\02_2020.tif",
        "E:\\Internship_Harvesting\\Dataset\\03_2020.tif"]

In [74]:
path_results = 'E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\'

# doing for all the bands for image 10_2019
### also saving the masked image band as a separate image, therefore we will have 30 masked images, 5 for each input image

In [91]:
input_image = gdal.Open(image[0])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I1B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F8FA72BA0> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


# doing for all the bands for image 11_2019

In [92]:
input_image = gdal.Open(image[1])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I2B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002105CCC5840> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


# doing for all bands for image 12_2019

In [93]:
input_image = gdal.Open(image[2])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I3B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F8EB431E0> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


# doing for all bands of image 01_2020

In [94]:
input_image = gdal.Open(image[3])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I4B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002105CCC5F00> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


# doing for all bands of image 02_2020

In [95]:
input_image = gdal.Open(image[4])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I5B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002105CCC59C0> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


# doing for all bands of image 03_2020

In [96]:
input_image = gdal.Open(image[5])
print(input_image)
    
image_zero = np.zeros((input_image.RasterYSize, input_image.RasterXSize, input_image.RasterCount))

for b in range(image_zero.shape[2]):
    image_zero[:, :, b] = input_image.GetRasterBand(b + 1).ReadAsArray()
    
    print(b)
        
    for i in range(polytoras_0_index.shape[0]):
        image_zero[:, :, b][polytoras_0_index[i,0], polytoras_0_index[i,1]] = np.NaN
        
    print(np.count_nonzero(np.isnan(image_zero[:, :, b])))
    
    io.imsave(path_results+'I6B{:d}.tif'.format(b+1), image_zero[:, :, b])
    
print(np.count_nonzero(np.isnan(image_zero)))

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002105CCC5570> >
0
40574909
1
40574909
2
40574909
3
40574909
4
40574909
202874545


### this completes the masking of the images, which will be used as the input data for classification process